# 债券基金选择

1. 获取长债基金基本信息
2. 获取基金历史行情，并进行年化收益率、夏普比率、最大回撤计算
3. 将计算结果进行标准化处理，并加权求和，得到最终得分
4. 根据得分进行排序，得到最终结果

In [62]:
# 导入库
import akshare as ak
import empyrical as ep
import pandas as pd
import datetime

## 1.获取长债基金基本信息

In [63]:
# 获取基金名称
fund_purchase_em_df = ak.fund_purchase_em()
# 获取基金类型为债券型-长债的基金
fund_purchase_em_bond_df = fund_purchase_em_df[fund_purchase_em_df['基金类型'].str.contains('债券型-长债')]
# 获取申购状态为开放申购的基金
fund_purchase_em_bond_open_df = fund_purchase_em_bond_df[fund_purchase_em_bond_df['申购状态'] == '开放申购']
# 删除基金简称结尾为C的基金
fund_purchase_em_bond_open_df = fund_purchase_em_bond_open_df[~fund_purchase_em_bond_open_df['基金简称'].str.endswith('C')]
# 获取基金代码
fund_code = fund_purchase_em_bond_open_df['基金代码'].reset_index(drop=True)

In [64]:
fund_purchase_em_bond_open_df

,序号,基金代码,基金简称,基金类型,最新净值/万份收益,最新净值/万份收益-报告时间,申购状态,赎回状态,下一开放日,购买起点,日累计限定金额,手续费
11,12,000015,华夏纯债债券A,债券型-长债,1.3000,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.08
16,17,000024,大摩双利增强债券A,债券型-长债,1.1303,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.08
22,23,000032,易方达信用债债券A,债券型-长债,1.1088,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.08
35,36,000053,鹏华永诚一年定开债券,债券型-长债,1.0256,04-14,开放申购,暂停赎回,2023-03-27,10.0,1.000000e+11,0.08
66,67,000086,南方稳利1年持有期债券A,债券型-长债,1.0797,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+10,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...
18172,18173,970038,东莞德益6个月持有债,债券型-长债,1.0636,04-14,开放申购,开放赎回,NaT,100.0,1.000000e+11,0.60
18193,18194,970059,安信瑞盈3个月滚动持有债B,债券型-长债,1.0577,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.05
18205,18206,970071,第一创业创享纯债,债券型-长债,1.0366,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.04
18240,18241,970106,第一创业创和一个月持有债,债券型-长债,1.0250,04-14,开放申购,开放赎回,NaT,10.0,1.000000e+11,0.04


## 2. 获取基金历史行情，并进行年化收益率、夏普比率、最大回撤计算

In [67]:
# 生成一个空df，列名为：基金代码，近一年收益率，成立以来收益率，近一年夏普比率，成立以来夏普比率，成立以来最大回撤
sharpe_ratio_all_df = pd.DataFrame(columns=['基金代码','近一年收益率','成立以来收益率','近一年夏普比率','成立以来夏普比率','成立以来最大回撤'])
today = datetime.date.today()
# 循环fund_code获取每个基金的指标数据
for i in range(len(fund_code)):
    if i % 100 == 0:
        print(i)
    try:
        fund_open_fund_info_em_df = ak.fund_open_fund_info_em(fund=fund_code[i])
        # 如果基金成立日期小于2020年1月1日，则计算夏普比率
        if fund_open_fund_info_em_df['净值日期'].min() < datetime.date(2020,1,1):
            # 将净值日期转换为datetime类型
            fund_open_fund_info_em_df['净值日期'] = pd.to_datetime(fund_open_fund_info_em_df['净值日期'])
            # 将净值日期设置为index
            fund_open_fund_info_em_df.set_index('净值日期', inplace=True)
            returns = fund_open_fund_info_em_df['日增长率']/100
            # 计算近一年年化收益率
            returns_1year = ep.stats.annual_return(returns[today - datetime.timedelta(days=365*5):today])
            # 计算成立以来年化收益率
            returns_all = ep.stats.annual_return(returns)
            # 计算夏普比率
            if returns[today - datetime.timedelta(days=365):today].std() == 0: #防止除数为0
                sharpe_ratio_year = 0
            else:
                sharpe_ratio_year = returns[today - datetime.timedelta(days=365):today].mean() / returns[today - datetime.timedelta(days=365):today].std()
            if returns.std() == 0:
                sharpe_ratio_all = 0
            else:
                sharpe_ratio_all = returns.mean() / returns.std()
            # 计算近5年最大回撤,因为10年前最大回撤有重大波动，所以只计算近5年的最大回撤
            max_drawdown = ep.stats.max_drawdown(returns[today - datetime.timedelta(days=365*5):today])
            sharpe_ratio_all_df.loc[i] = [fund_code[i],returns_1year,returns_all,sharpe_ratio_year,sharpe_ratio_all,max_drawdown]
    except:
        continue

0
100
200
300
400
500
600


## 3. 将计算结果进行标准化处理，并加权求和，得到最终得分

In [84]:
# 用于保存数据
sharpe_ratio_df = sharpe_ratio_all_df.copy()
# 删除近一年收益率超过0.1的数据，改收益率源于巨额赎回导致的基金净值异常
sharpe_ratio_df = sharpe_ratio_df[sharpe_ratio_df['近一年收益率'] < 0.1]
#创建列，在sharpe_ratio_df中，将近一年收益率，成立以来收益率，近一年夏普比率，成立以来夏普比率，成立以来最大回撤进行标准化处理，处理公式为：(x - mean) / std
sharpe_ratio_df['近一年收益率_norm'] = (sharpe_ratio_df['近一年收益率'] - sharpe_ratio_df['近一年收益率'].mean()) / sharpe_ratio_df['近一年收益率'].std()
sharpe_ratio_df['成立以来收益率_norm'] = (sharpe_ratio_df['成立以来收益率'] - sharpe_ratio_df['成立以来收益率'].mean()) / sharpe_ratio_df['成立以来收益率'].std()
sharpe_ratio_df['近一年夏普比率_norm'] = (sharpe_ratio_df['近一年夏普比率'] - sharpe_ratio_df['近一年夏普比率'].mean()) / sharpe_ratio_df['近一年夏普比率'].std()
sharpe_ratio_df['成立以来夏普比率_norm'] = (sharpe_ratio_df['成立以来夏普比率'] - sharpe_ratio_df['成立以来夏普比率'].mean()) / sharpe_ratio_df['成立以来夏普比率'].std()
# 因为最大回撤越小越好，所以需要将最大回撤进行反向处理
sharpe_ratio_df['成立以来最大回撤_norm'] = - (sharpe_ratio_df['成立以来最大回撤'] - sharpe_ratio_df['成立以来最大回撤'].mean()) / sharpe_ratio_df['成立以来最大回撤'].std()
# 加权进行指标计算，权重为：近一年收益率*0.2，成立以来收益率*0.4，近一年夏普比率*0.1，成立以来夏普比率*0.2，成立以来最大回撤*0.2
sharpe_ratio_df['综合指标'] = sharpe_ratio_df['近一年收益率']*0.2 + sharpe_ratio_df['成立以来收益率']*0.4 + sharpe_ratio_df['近一年夏普比率']*0.1 + sharpe_ratio_df['成立以来夏普比率']*0.2 + sharpe_ratio_df['成立以来最大回撤']*0.2
# 将sharpe_ratio_df添加一列基金简称，根据每行的基金代码，找到在 fund_purchase_em_bond_open_df对应的基金简称，下一开放日，日累计限定金额，手续费
sharpe_ratio_df['基金简称'] = sharpe_ratio_df['基金代码'].apply(lambda x: fund_purchase_em_bond_open_df[fund_purchase_em_bond_open_df['基金代码'] == x]['基金简称'].values[0])
sharpe_ratio_df['下一开放日'] = sharpe_ratio_df['基金代码'].apply(lambda x: fund_purchase_em_bond_open_df[fund_purchase_em_bond_open_df['基金代码'] == x]['下一开放日'].values[0])
sharpe_ratio_df['日累计限定金额'] = sharpe_ratio_df['基金代码'].apply(lambda x: fund_purchase_em_bond_open_df[fund_purchase_em_bond_open_df['基金代码'] == x]['日累计限定金额'].values[0])
sharpe_ratio_df['手续费'] = sharpe_ratio_df['基金代码'].apply(lambda x: fund_purchase_em_bond_open_df[fund_purchase_em_bond_open_df['基金代码'] == x]['手续费'].values[0])

## 4. 将计算结果进行排序，得到最终结果

In [89]:
# 将sharpe_ratio_df按照综合指标降序排列,提取出前20个基金
best_fund_code = sharpe_ratio_df.sort_values(by='综合指标',ascending=False).head(20)

In [90]:
best_fund_code

,基金代码,近一年收益率,成立以来收益率,近一年夏普比率,成立以来夏普比率,成立以来最大回撤,近一年收益率_norm,成立以来收益率_norm,近一年夏普比率_norm,成立以来夏普比率_norm,成立以来最大回撤_norm,综合指标,基金简称,下一开放日,日累计限定金额,手续费
64,003078,0.043110,0.042011,0.417035,0.430785,-0.015367,0.596983,0.378419,2.763188,2.629874,-0.555934,0.150213,泰康安惠纯债债券A,NaT,1.000000e+11,0.08
155,005872,0.030160,0.030160,0.303596,0.482508,-0.006285,-0.451580,-0.534759,1.505542,3.229729,-0.909376,0.143700,太平恒利纯债,NaT,1.000000e+11,0.08
118,004629,0.033681,0.034844,0.315045,0.464190,-0.010697,-0.166478,-0.173866,1.632474,3.017288,-0.737680,0.142877,国寿安保安瑞纯债债券,NaT,1.000000e+11,0.08
132,005315,0.036697,0.038523,0.326935,0.406589,-0.011219,0.077749,0.109685,1.764290,2.349264,-0.717376,0.134516,泰达宏利交利3个月定开债A,2023-04-14,0.000000e+00,0.80
86,003614,0.035821,0.037265,0.285878,0.432124,-0.014314,0.006823,0.012706,1.309109,2.645410,-0.596915,0.134220,信诚景瑞债券A,NaT,1.000000e+11,0.08
194,006681,0.031133,0.031133,0.296366,0.426711,-0.011187,-0.372798,-0.459786,1.425385,2.582626,-0.718598,0.131421,景顺长城景泰聚利纯债,NaT,0.000000e+00,0.80
88,003668,0.037518,0.037905,0.304233,0.391075,-0.007973,0.144175,0.061999,1.512600,2.169342,-0.843691,0.129709,东方红益鑫纯债A,NaT,1.000000e+11,0.07
280,007703,0.030679,0.030679,0.322854,0.405580,-0.010792,-0.409533,-0.494745,1.719040,2.337572,-0.733983,0.129651,万家鑫盛纯债A,NaT,0.000000e+00,0.80
139,005531,0.038567,0.039298,0.279694,0.378142,-0.016173,0.229135,0.169332,1.240545,2.019353,-0.524585,0.123796,华安安悦债券A,NaT,0.000000e+00,0.80
168,006152,0.029503,0.029503,0.294502,0.381801,-0.011439,-0.504782,-0.585389,1.404715,2.061788,-0.708784,0.121224,国联安增鑫纯债A,NaT,1.000000e+11,0.06


其他不足与思考：
1. 是否应对基金规模进行上下限限制
2. 无法获取基金规模变动情况，对于基金规模变动较大的基金，是否应该进行排除
3. 对于综合指标的权重分配是否合理，是否应该对权重进行调整

In [ ]:
#长债基金选择英文版
